In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get POSTGRES_ADDRESS from .env file
POSTGRES_ADDRESS = os.getenv("POSTGRES_ADDRESS")

# Create engine with the address
engine = create_engine(POSTGRES_ADDRESS)

df = pd.read_sql_query("SELECT * FROM benchmark_runs_v2", engine)
# Set pandas display options to make tables wider
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Set width to 1000 characters
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple lines
# Drop the 'args' column if it exists in the DataFrame
# if 'args' in df.columns:
df = df.drop(columns=['config_details_json'])
print(df.columns)


Index(['id', 'timestamp', 'input_file', 'output_dir', 'notes', 'duplicate_count', 'record_count', 'implementation', 'num_nodes', 'threshold', 'ngram_size', 'min_ngram_size', 'num_perm', 'execution_time', 'limit_files', 'total_size_gb', 'metrics'], dtype='object')


In [3]:
df.metrics.tolist()

[{'nd_metrics': {'duplicate_count': 16056,
   'execution_time': 84.51909303665161,
   'false_positive_rate': 0.43359419653890957},
  'cl_metrics': [{'inference_time': 0.0,
    'train_time': 0.0,
    'total_time': 2.12062406539917,
    'stage': 'stage1'},
   {'inference_time': 11.204771542549134,
    'train_time': 14.100396943092345,
    'total_time': 89.97783589363098,
    'stage': 'stage2'}]}]

In [ ]:

df.iloc[1:].to_csv("benchmark_runs.csv", index=False)

In [ ]:
df.execution_time.sum()/3600

In [ ]:
df.duplicate_count+df.record_count

In [ ]:
2.7*60

In [ ]:
752.379005/

In [ ]:
421_609

In [ ]:
13831082

In [ ]:
13_831_082

In [ ]:
200/

In [ ]:
1_749_756

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("/home/ohadr/database_project_c/benchmark_results.db")

df = pd.read_sql_query("SELECT * FROM benchmark_runs", conn)
df

In [ ]:
print(df[["duplicate_count","record_count","execution_time", "total_size_gb"]].to_markdown())